<h1>Example post quantum secure communication between ALice and Bob</h1>

1. Required prior knowledge of the buffer size used, for all the parties.

In [41]:
buffer_size = 3200 # y of (y X y)
sha_key_str = 'Tuesday Evening'

2. QuantCrypt provides Kyber1024 key encapsulation mechanism.

In [42]:
from quantcrypt.kem import Kyber

kem = Kyber()

3. Alice generates key pair.

In [43]:
public_key, secret_key = kem.keygen()

4. Bob generates cipher and shared secret using Alice's public key.

In [44]:
cipher_text, shared_secret = kem.encaps(public_key)


5. Alice decrypts cipher she receives from Bob to extract shared secret.

In [45]:

shared_secret_copy = kem.decaps(secret_key, cipher_text)


6. Bob encrypts the message to be sent using the shared_secret.

In [46]:
from selective_encryption import encrypt
from utils import time_it

encrypt(file_path='data/files/landscape.png',
           sha_key_str=sha_key_str, 
           buffer=buffer_size,
           aes_key=shared_secret)

itr_count: 0
dwt2d: 0.009071625
dwt2d: 0.002481209
sha256_with_key: 0.000349417
sha512_with_key: 0.001491584
xor_with_sha_key: 0.002172875
xor_with_sha_key: 0.008268792
aes_encrypt: 0.002706125
_se_encrypt: 0.032332209
itr_count: 1
dwt2d: 0.006889167
dwt2d: 0.002097375
sha256_with_key: 0.000308041
sha512_with_key: 0.001253792
xor_with_sha_key: 0.001767625
xor_with_sha_key: 0.007116500
aes_encrypt: 0.002516083
_se_encrypt: 0.027194459
encrypt: 0.065599417


7. Alice decrypts the message using shared_secret_copy.

In [47]:
from selective_encryption import decrypt

decrypt(file_PF= 'data/files/enc/landscape.png_PF.npy',
           file_PPF_1= 'data/files/enc/landscape.png_PPF_1.npy',
           file_PPF_2= 'data/files/enc/landscape.png_PPF_2.npy',
           buffer=buffer_size,
           output_file_path='data/files/dec/landscape.png',
           sha_key_str=sha_key_str, 
           aes_key=shared_secret_copy)

aes_decrypt: 0.005290458
sha256_with_key: 0.000893792
xor_with_sha_key: 0.007387000
sha512_with_key: 0.002393250
xor_with_sha_key: 0.014319833
idwt2d: 0.003614459
idwt2d: 0.010858334
aes_decrypt: 0.002106791
sha256_with_key: 0.000264500
xor_with_sha_key: 0.001763625
sha512_with_key: 0.001162958
xor_with_sha_key: 0.006826834
idwt2d: 0.003081875
idwt2d: 0.011058750
decrypt: 2.373859500
